In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.spatial import distance

# Loads each whole scenario as a sheet

In [ ]:
start_folder = 'AllCSVs'
csvs_scen = glob.glob(r'{}\*.csv'.format(start_folder))
df_scen = []

for c in csvs_scen:
    df = pd.read_csv(c).drop(['type','messageID','spd_noise0','spd_noise1','spd_noise2' ,'pos_noise0' ,'pos_noise1' ,'pos_noise2', 'spd2', 'pos2'], axis=1)
    df_scen.append(df)
print(len(df_scen))
df_scen[0]

In [ ]:
# Define a function to find the two closest rows above a given row based on A and B values
def find_closest_rows_above(df, row):
    # select a row to find closest rows to
    given_row = row

    # Find the index of the given row
    row_index = df.index.get_loc(given_row.name)

    # Slice the DataFrame to only consider rows above the given row
    above_df = df.iloc[max(0, row_index-2000):row_index]

    if len(above_df) == 0:
        return pd.DataFrame()

    # select the A and B columns of the DataFrame
    df_ab = above_df[['time', 'pos0', 'pos1']]

    # calculate the Euclidean distance between the given row and all other rows based on A and B values
    distances = distance.cdist(df_ab, [given_row[['time', 'pos0', 'pos1']]], metric='euclidean')

    # get the indices of rows sorted by distance
    sorted_indices = distances.argsort(axis=0).flatten()

    # get the closest rows as a DataFrame
    closest_rows = df.iloc[sorted_indices]
    
    # Create a Boolean mask to select the rows that contain the value. Ensures that senders are not the same
    mask = closest_rows['sender'] == search_row.sender

    # Drop the rows that match the mask
    closest_rows.drop(closest_rows[mask].index, inplace=True)

    return closest_rows

In [ ]:
df_closest = []
p = 0
for df in df_scen:
    df_copy = df.copy()
    for i in range(2, len(df_copy)): #len(df)
#         print(i)
#         display(search_row)
        search_row = df_copy.iloc[i] 
        closest_rows = find_closest_rows_above(df.copy(), search_row).reset_index()
        closest_car_nums = [0,1]
        feats = ['pos0','pos1','spd0','spd1']
#         display(closest_rows)
        for num in closest_car_nums:
            for f in feats:
                df_copy.loc[i, f'{f}Close{num}'] = closest_rows.loc[num, f]
    
    df_copy.to_csv(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ClosestCarCSVsModel2\scenario_{}.csv'.format(p))  
    df_closest.append(df_copy)
    p+=1

In [ ]:
display(df_closest[1])

In [ ]:
# Next Step **Chop out the first 2 rows of the df**
start_folder = 'AllCSVs'
csvs = glob.glob(r'{}\*.csv'.format(start_folder))
df_list = []

for c in csvs:
    df = pd.read_csv(c)
    for i in np.unique(df.sender):
        df_list.append(df[df.sender.isin([i])].copy())
    #     df[df.ID.isin([subs[0]])&df.Vigil.isin(vigils[0])].copy()
print(len(df_list))
# df_list[0]

# ------------------------------------------Template Code-----------------------------------------

In [ ]:
import pandas as pd
from scipy.spatial import distance

# create a sample DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 4,6,7,8,9],
    'B': [2, 4, 6, 8, 7,3,7,3,8],
    'C': [3, 6, 9, 12, 3,2,9,6,4]
})

# select a row to find closest rows to
given_row = df.iloc[4]

# Find the index of the given row
row_index = df.index.get_loc(given_row.name)

# Slice the DataFrame to only consider rows above the given row
above_df = df.iloc[:row_index]

# if len(above_df) == 0:
#     return pd.DataFrame()


# select the A and B columns of the DataFrame
df_ab = above_df[['A', 'B']]

# calculate the Euclidean distance between the given row and all other rows based on A and B values
distances = distance.cdist(df_ab, [given_row[['A', 'B']]], metric='euclidean')

# get the indices of rows sorted by distance
sorted_indices = distances.argsort(axis=0).flatten()

# select the two closest rows (excluding the given row itself)
closest_indices = sorted_indices[0:3]

# get the closest rows as a DataFrame
closest_rows = df.iloc[closest_indices]

print(closest_rows)

In [ ]:
import pandas as pd
from scipy.spatial.distance import cdist

# Generate a sample DataFrame
df = pd.DataFrame({'A': [1, 1, 1, 4, 1],
                   'B': [6, 2, 2, 9, 2],
                   'C': [11, 3, 3, 14, 3]})

# Define a function to find the two closest rows above a given row based on A and B values
def find_closest_rows_above(df, row):
    # Find the index of the given row
    row_index = df.index.get_loc(row.name)
    
    # Slice the DataFrame to only consider rows above the given row
    above_df = df.iloc[:row_index]
    
    if len(above_df) == 0:
        return pd.DataFrame()
    
    # Compute the distances between the given row and all rows above it in the DataFrame based on A and B values
    distances = cdist(above_df[['A', 'B']].values, row[['A', 'B']].values.reshape(1, -1))
    
    # Sort the distances in ascending order and retrieve the index of the closest row
    closest_index = distances.argmin(axis=0)[0]
    
    # Use the index to retrieve the closest row from the DataFrame
    closest_row = above_df.iloc[closest_index]
    
    # Find the index of the second closest row
    second_closest_index = (distances != distances[closest_index]).argmax(axis=0)[0]
    
    # Use the index to retrieve the second closest row from the DataFrame
    second_closest_row = above_df.iloc[second_closest_index]
    
    return pd.concat([closest_row, second_closest_row], axis=1).T

# Define a sample row to search for
search_row = df.iloc[2] 
# For i in range(0, len(df)): 
#     search_row = df.iloc[i]
# Append the closest_rows to a new dataframe and then glue the 2 dataframes together at the end




# Find the two closest rows above the search row based on A and B values
closest_rows = find_closest_rows_above(df, search_row)

print(closest_rows)

In [ ]:
df

In [ ]:
import pandas as pd

# create a DataFrame
df = pd.DataFrame({'pos1': [25, 30, 35],
                   'pos2': [25, 30, 35],
                   'pos3': [25, 30, 35]})

# create a new DataFrame with the new columns
closest_rows = pd.DataFrame({'pos1DIFF': [50000, 60000, 70000],
                         'pos2DIFF': [1, 2, 3]})

# concatenate the new DataFrame to the existing one
result = pd.concat([df, closest_rows], axis=1)

print(result)

In [ ]:
X = []
y = []
for i in df_normal:
    X.append(np.asarray(df_normal[0].iloc[0:10,2:]))
    y.append(np.asarray(df_normal[0].iloc[10,2:]))
X = np.asarray(X)
y = np.asarray(y)
np.save(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\X.npy'.format(start_folder), X)
np.save(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\y.npy'.format(start_folder), y)